In [1]:
! pip install autogen-agentchat "autogen-ext[ollama]"

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.ollama import OllamaChatCompletionClient

In [3]:
ollama_client = OllamaChatCompletionClient(model="llama3")  

Agent Team

In [4]:
# Planner Agent (orchestrates the task)
planner = AssistantAgent(
    name="planner",
    system_message=(
        "You are the Planner Agent. "
        "Break the user's request into subtasks and coordinate with the coder. "
        "Stop when the answer is complete."
    ),
    model_client=ollama_client
)

In [5]:
# Coder / Expert Agent (executes subtasks)
coder = AssistantAgent(
    name="coder",
    system_message=(
        "You are the Coder Agent. "
        "You respond to questions, produce code, or solve subtasks assigned by the Planner. "
        "Return only useful outputs."
    ),
    model_client=ollama_client
)

In [6]:
# Review Agent (reviews results)
critic = AssistantAgent(
    name="critic",
    system_message=(
        "You are the Review Agent. "
        "You review answers and request improvements. "
        "Reply only 'APPROVE' if the task is done."
    ),
    model_client=ollama_client,
)

In [7]:
# The termination condition is a text termination, which will cause the chat to terminate when the text "APPROVE" is received.
termination = TextMentionTermination("APPROVE")

In [8]:
group_chat = RoundRobinGroupChat([planner, coder, critic], termination_condition=termination, max_turns=5)

In [9]:
# Multi-agent task
task = (
    "Write a short, clear explanation (one paragraph) of the advantages of electric vehicles. "
    "The Planner should coordinate with the Coder to produce the final answer. "
    "The Critic should review the results."
)

In [10]:
# Run non-streaming
response = await group_chat.run(task=task)
print("Response:", response)

Response: messages=[TextMessage(id='375ed05e-50c2-48cd-8551-385f68814f3f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 18, 15, 45, 40, 413047, tzinfo=datetime.timezone.utc), content='Write a short, clear explanation (one paragraph) of the advantages of electric vehicles. The Planner should coordinate with the Coder to produce the final answer. The Critic should review the results.', type='TextMessage'), TextMessage(id='c3437133-53eb-4111-8ca6-764f9c605a1b', source='planner', models_usage=RequestUsage(prompt_tokens=81, completion_tokens=374), metadata={}, created_at=datetime.datetime(2025, 11, 18, 15, 46, 2, 640260, tzinfo=datetime.timezone.utc), content="I'll start by breaking down the request into subtasks:\n\n1. Identify the main advantages of electric vehicles\n2. Write a clear and concise paragraph explaining these benefits\n3. Coordinate with the Coder to generate the paragraph in a visually appealing format (e.g., bold headings, bullet po

In [11]:
def format_textmessages(messages):
    summary = "\n### Conversation Summary\n\n"
    
    for i, msg in enumerate(messages, start=1):
        source = msg.source.capitalize()
        content = msg.content.strip()
        
        if source == "User":
            summary += f"**{i}. User Message**\n> {content}\n\n"
        elif source == "Planner":
            summary += f"**{i}. Planner Message**\n- Draft/Coordination:\n{content}\n\n"
        elif source == "Coder":
            summary += f"**{i}. Coder Message**\n> {content}\n\n"
        elif source == "Critic":
            summary += f"**{i}. Critic Message**\n> {content} ✅\n\n"
        else:
            summary += f"**{i}. {source} Message**\n> {content}\n\n"
    
    return summary

# Print formatted summary
formatted_summary = format_textmessages(response.messages)
print(formatted_summary)


### Conversation Summary

**1. User Message**
> Write a short, clear explanation (one paragraph) of the advantages of electric vehicles. The Planner should coordinate with the Coder to produce the final answer. The Critic should review the results.

**2. Planner Message**
- Draft/Coordination:
I'll start by breaking down the request into subtasks:

1. Identify the main advantages of electric vehicles
2. Write a clear and concise paragraph explaining these benefits
3. Coordinate with the Coder to generate the paragraph in a visually appealing format (e.g., bold headings, bullet points)
4. Review the results with the Critic to ensure accuracy and completeness

Here's my attempt at writing the explanation:

**Advantages of Electric Vehicles**

Electric vehicles offer numerous advantages over traditional gasoline-powered cars. **Zero Emissions**: EVs produce no tailpipe emissions, reducing greenhouse gas emissions and air pollution in urban areas. **Lower Operating Costs**: Electric vehic